# Evaluate model endpoints using Prompt Flow Eval APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [11]:
%pip install azure-ai-evaluation
%pip install promptflow-azure

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\sydneylister\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\sydneylister\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Parameters and imports

In [19]:
from pprint import pprint

import pandas as pd
import random

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [20]:
env_var = {
    "gpt4-0613": {
        "endpoint": "https://ai-***.**.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2023-03-15-preview",
        "key": "***",
    },
    "gpt35-turbo": {
        "endpoint": "https://ai-**.openai.azure.com/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-03-15-preview",
        "key": "***",
    },
    "mistral7b": {
        "endpoint": "https://***.eastus.inference.ml.azure.com/v1/chat/completions",
        "key": "***",
    },
    "phi3_mini_serverless": {
        "endpoint": "https://Phi-3-mini-4k-instruct-rpzhe.eastus2.models.ai.azure.com/v1/chat/completions",
        "key": "***",
    },
    "tiny_llama": {
        "endpoint": "https://api-inference.huggingface.co/models/TinyLlama/TinyLlama-1.1B-Chat-v1.0/v1/chat/completions",
        "key": "***",
    },
    "gpt2": {
        "endpoint": "https://api-inference.huggingface.co/models/openai-community/gpt2",
        "key": "***",
    },
}


Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [21]:
azure_ai_project = {"subscription_id": "<your-subscription-id>", "resource_group_name": "<your-resource-group-name>", "project_name": "<your-project-name>"}

In [22]:
import os

# Use the following code to set the environment variables if not already set. If set, you can skip this step.

os.environ["AZURE_OPENAI_API_KEY"] = "<your-api-key>"
os.environ["AZURE_OPENAI_API_VERSION"] = "<api version>"
os.environ["AZURE_OPENAI_DEPLOYMENT"] = "<your-deployment>"
os.environ["AZURE_OPENAI_ENDPOINT"] = "<your-endpoint>"

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [23]:
df = pd.read_json("data.jsonl", lines=True)
print(df.head())

                                        question  \
0                 What is the capital of France?   
1             Which tent is the most waterproof?   
2           Which camping table is the lightest?   
3  How much does TrailWalker Hiking Shoes cost?    

                                             context  \
0                   France is the country in Europe.   
1  #TrailMaster X4 Tent, price $250,## BrandOutdo...   
2  #BaseCamp Folding Table, price $60,## BrandCam...   
3  #TrailWalker Hiking Shoes, price $110## BrandT...   

                                        ground_truth  
0                                              Paris  
1  The TrailMaster X4 tent has a rainfly waterpro...  
2  The BaseCamp Folding Table has a weight of 15 lbs  
3    The TrailWalker Hiking Shoes are priced at $110  


## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [24]:
model_config = {
        "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
        "api_key": os.environ.get("AZURE_OPENAI_KEY"),
        "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    }

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [25]:
from app_target import ModelEndpoints
import pathlib

from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    ContentSafetyEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)


content_safety_evaluator = ContentSafetyEvaluator(azure_ai_project)
relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)

models = [
    "gpt4-0613",
    "gpt35-turbo",
    "mistral7b",
    "phi3_mini_serverless",
    "tiny_llama",
    "gpt2",
]

path = str(pathlib.Path(pathlib.Path.cwd())) + "/data.jsonl"

for model in models:
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        evaluation_name="Eval-Run-" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "content_safety": content_safety_evaluator,
            "coherence": coherence_evaluator,
            "relevance": relevance_evaluator,
            "groundedness": groundedness_evaluator,
            "fluency": fluency_evaluator,
            "similarity": similarity_evaluator,
        },
        evaluator_config={
            "content_safety": {"query": "${data.question}", "response": "${target.response}"},
            "coherence": {"response": "${target.response}", "query": "${data.question}"},
            "relevance": {"response": "${target.query}", "context": "${data.context}", "query": "${data.question}"},
            "groundedness": {
                "response": "${target.response}",
                "context": "${data.context}",
                "query": "${data.question}",
            },
            "fluency": {"response": "${target.response}", "context": "${data.context}", "query": "${data.question}"},
            "similarity": {"response": "${target.response}", "context": "${data.context}", "query": "${data.question}"},
        },
    )

[2024-09-26 16:53:21 -0700][promptflow][WARNING] - Found existing c:/azureai-samples/scenarios/evaluate-model-endpoints/flow.flex.yaml, will not respect it in runtime.


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=evaluate_model_endpoints_20240926_165321_352972


[2024-09-26 16:53:28 -0700][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run evaluate_model_endpoints_20240926_165321_352972, log path: C:\Users\sydneylister\.promptflow\.runs\evaluate_model_endpoints_20240926_165321_352972\logs.txt
[2024-09-26 16:53:47 -0700][promptflow._sdk._orchestrator.run_submitter][WARNING] - 4 out of 4 runs failed in batch run.
 Please check out C:/Users/sydneylister/.promptflow/.runs/evaluate_model_endpoints_20240926_165321_352972 for more details.


2024-09-26 16:53:28 -0700   39604 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2024-09-26 16:53:28 -0700   39604 execution          WARNING  Starting run without column mapping may lead to unexpected results. Please consult the following documentation for more information: https://aka.ms/pf/column-mapping
2024-09-26 16:53:28 -0700   39604 execution.bulk     INFO     Current system's available memory is 10048.08203125MB, memory consumption of current process is 372.0625MB, estimated available worker count is 10048.08203125/372.0625 = 27
2024-09-26 16:53:28 -0700   39604 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 4, 'estimated_worker_count_based_on_memory_usage': 27}.
2024-09-26 16:53:37 -0700   39604 execution.bulk     INFO     Process name(SpawnProcess-32)-Process id(43356)-Line number(0) start execution.
2024-09-26 16:53:3

EvaluationException: Missing required inputs for evaluator content_safety : ['response'].

View the results

In [ ]:
pprint(results)

In [ ]:
pd.DataFrame(results["rows"])